In [7]:
import pandas as pd
import numpy as np
from utils import get_species, get_labels, get_labels_all

In [38]:
X, y, y_all = get_species(), get_labels(), get_labels_all()

In [39]:
X = X[list(filter(lambda x : "virus" not in x and "unclassified" not in x, X.columns))]
X.shape

(12532, 1392)

In [40]:
weights = ((y_all == "T2D") | (y_all == "Obesity") | (y_all == "Overweight") | (y_all == "Underweight")).values

In [41]:
c = 0.00001
X, y = X.iloc[~weights, :], y.iloc[~weights, :]

In [42]:
X = X.drop(columns=["UNKNOWN"])
X

k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-23  SAMEA3879547                                               0.008611                                                                                                 
         SAMEA3879551                                               0.000000                                                                                                 
         SAMEA3879543                                               0.001520                                                                                                 
         SAMEA3879565                                               0.003630                                                                                                 
         SAMEA3879546                                               0.000012                                                                                                 
...                                                                      ...                                                                                                 
P140     SAMN07509557                                               0.000000                                                                                                 
         SAMN07509558                                               0.000000                                                                                                 
         SAMN07509546                                               0.000000                                                                                                 
         SAMN07509552                                               0.000000                                                                                                 
         SAMN07509921                                               0.000000                                                                                                 

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera|s__Methanosphaera_stadtmanae  \
Study_ID Sample Accession                                                                                                                                               
GMHI-23  SAMEA3879547                                                    0.0                                                                                            
         SAMEA3879551                                                    0.0                                                                                            
         SAMEA3879543                                                    0.0                                                                                            
         SAMEA3879565                                                    0.0                                                                                            
         SAMEA3879546                                                    0.0                                                                                            
...                                                                      ...                                                                                            
P140     SAMN07509557                                                    0.0                                                                                            
         SAMN07509558                                                    0.0                                                                                            
         SAMN07509546                                                    0

In [43]:
X = np.divide(X, X.values.sum(axis=1, keepdims=True))

In [44]:
X = X.fillna(0)

In [45]:
r = np.sum(X > c, axis=1)
r

Study_ID  Sample Accession
GMHI-23   SAMEA3879547         88
          SAMEA3879551         80
          SAMEA3879543        105
          SAMEA3879565         99
          SAMEA3879546         66
                             ... 
P140      SAMN07509557         43
          SAMN07509558          0
          SAMN07509546         32
          SAMN07509552         37
          SAMN07509921         53
Length: 9661, dtype: int64

In [46]:
inv_sim = 1 / np.sum(X ** 2, axis=1)
inv_sim = inv_sim.replace([np.inf, -np.inf], 0)
inv_sim

Study_ID  Sample Accession
GMHI-23   SAMEA3879547        13.275839
          SAMEA3879551        11.567633
          SAMEA3879543        17.382471
          SAMEA3879565        13.749011
          SAMEA3879546         8.465760
                                ...    
P140      SAMN07509557         5.780861
          SAMN07509558         0.000000
          SAMN07509546         1.349734
          SAMN07509552         4.327599
          SAMN07509921         7.105770
Length: 9661, dtype: float64

In [47]:
shannon = np.sum(X * np.log(X), axis=1)
shannon

Study_ID  Sample Accession
GMHI-23   SAMEA3879547       -3.200817
          SAMEA3879551       -2.939579
          SAMEA3879543       -3.414508
          SAMEA3879565       -3.289911
          SAMEA3879546       -2.633228
                                ...   
P140      SAMN07509557       -2.274336
          SAMN07509558        0.000000
          SAMN07509546       -0.790609
          SAMN07509552       -1.871630
          SAMN07509921       -2.449154
Length: 9661, dtype: float64

In [48]:
X = X > c
X['richness'] = r
X['shannon'] = shannon
X['inv_sim'] = inv_sim
X

k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-23  SAMEA3879547                                                   True                                                                                                 
         SAMEA3879551                                                  False                                                                                                 
         SAMEA3879543                                                   True                                                                                                 
         SAMEA3879565                                                   True                                                                                                 
         SAMEA3879546                                                   True                                                                                                 
...                                                                      ...                                                                                                 
P140     SAMN07509557                                                  False                                                                                                 
         SAMN07509558                                                  False                                                                                                 
         SAMN07509546                                                  False                                                                                                 
         SAMN07509552                                                  False                                                                                                 
         SAMN07509921                                                  False                                                                                                 

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera|s__Methanosphaera_stadtmanae  \
Study_ID Sample Accession                                                                                                                                               
GMHI-23  SAMEA3879547                                                  False                                                                                            
         SAMEA3879551                                                  False                                                                                            
         SAMEA3879543                                                  False                                                                                            
         SAMEA3879565                                                  False                                                                                            
         SAMEA3879546                                                  False                                                                                            
...                                                                      ...                                                                                            
P140     SAMN07509557                                                  False                                                                                            
         SAMN07509558                                                  False                                                                                            
         SAMN07509546                                                  Fal

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
from sklearn.metrics import balanced_accuracy_score, accuracy_score

In [60]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
richness_scaled_train = s.fit_transform(X_train[['richness']])
richness_scaled_test = s.transform(X_test[['richness']])
is_scaled_train = s.fit_transform(X_train[['inv_sim']])
is_scaled_test = s.transform(X_test[['inv_sim']])
shannon_scaled_train = s.fit_transform(X_train[['shannon']])
shannon_scaled_test = s.transform(X_test[['shannon']])

In [61]:
X_train['richness'] = richness_scaled_train
X_test['richness'] = richness_scaled_test
X_train['shannon'] = shannon_scaled_train
X_test['shannon'] = shannon_scaled_test
X_train['inv_sim'] = is_scaled_train
X_test['inv_sim'] = is_scaled_test

In [62]:
X_test

k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
P86      SAMEA5335524                                                  False                                                                                                 
P136     SAMEA104416221                                                False                                                                                                 
GMHI-32  SAMN05414951                                                   True                                                                                                 
P77      SAMEA4719765                                                  False                                                                                                 
P2       SAMEA4545381                                                   True                                                                                                 
...                                                                      ...                                                                                                 
         SAMEA4545332                                                   True                                                                                                 
P81      SAMEA4968269                                                  False                                                                                                 
GMHI-4   SAMN00715178                                                  False                                                                                                 
P74      SAMEA7045595                                                  False                                                                                                 
GMHI-13  SAMN06110452                                                  False                                                                                                 

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera|s__Methanosphaera_stadtmanae  \
Study_ID Sample Accession                                                                                                                                               
P86      SAMEA5335524                                                  False                                                                                            
P136     SAMEA104416221                                                False                                                                                            
GMHI-32  SAMN05414951                                                  False                                                                                            
P77      SAMEA4719765                                                  False                                                                                            
P2       SAMEA4545381                                                  False                                                                                            
...                                                                      ...                                                                                            
         SAMEA4545332                                                  False                                                                                            
P81      SAMEA4968269                                                  False                                                                                            
GMHI-4   SAMN00715178                                                  Fal

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score

clf = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=1, class_weight="balanced")
clf.fit(X_train.iloc[:, :-3], y_train)
y_hat = clf.predict(X_test.iloc[:, :-3])
balanced_accuracy_score(y_test, y_hat), accuracy_score(y_test, y_hat)

/Users/daniel/opt/anaconda3/envs/data/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.7723926341837184, 0.7734092084842215)